In [ ]:
import requests
from tqdm import tqdm
url = "https://hacker-news.firebaseio.com/v0/user/alexmolas.json?print=pretty"
user = requests.get(url).json()

scores = []
for sub in tqdm(user["submitted"]):
    url = f"https://hacker-news.firebaseio.com/v0/item/{sub}.json"
    item = requests.get(url).json()
    if not item.get("deleted") and not item.get("dead") and item["type"] == "story":
        scores.append(item["score"])

In [1]:
import httpx
import asyncio
from tqdm import tqdm

async def fetch_user_data(url):
    async with httpx.AsyncClient() as client:
        response = await client.get(url)
    return response.json()

async def fetch_item_data(url, scores):
    async with httpx.AsyncClient() as client:
        response = await client.get(url)
    item = response.json()
    if not item.get("deleted") and not item.get("dead") and item["type"] == "story":
        scores.append(item["score"])



async def fetch_user_scores(url):
    user = await fetch_user_data(url)

    scores = []
    item_urls = [f"https://hacker-news.firebaseio.com/v0/item/{sub}.json" for sub in user["submitted"]]

    tasks = [fetch_item_data(url, scores) for url in item_urls]
    await asyncio.gather(*tasks)

    return scores


def get_user_scores(username):
    user_url = f"https://hacker-news.firebaseio.com/v0/user/{username}.json?print=pretty"

    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    scores = loop.run_until_complete(fetch_user_scores(user_url))

    loop.close()

    return scores

username = "alexmolas"
scores_result = get_user_scores(username)
print(f"{username}'s scores: {scores_result}")


RuntimeError: Cannot run the event loop while another loop is running

In [ ]:
item